Мы владеем сетью магазинов, в которых продаются различные товары.  

Наши сотрудники собрали датасет из 8523 продаж товаров в различных магазинах нашей сети. Наша задача построить модель, предсказывающую продажи каждого продукта в конкретном магазине.  

Используя эту модель, попробовать выявить факторы, больше всего влияющие на увеличение продаж.  

Описание датасета:

Variable	Description  
- Item_Identifier	Id продукта  
- Item_Weight	Вес продукта  
- Item_Fat_Content	Содержание жира в продукте  
- Item_Visibility	%полок, отведенный под наш продукт в конкретном магазине  
- Item_Type	Категория продукта  
- Item_MRP	Максимальная цена продажи продукта  
- Outlet_Identifier	Идентификатор магазина  
- Outlet_Establishment_Year	Год открытия магазина  
- Outlet_Size	Площадь магазина  
- Outlet_Location_Type	Тип города, в котором расположен магазин  
- Outlet_Type	Признак является ли магазин продуктовым или супермаркетом  
- Item_Outlet_Sales	Продажи продукта в конкретном магазине. Именно ее и надо предсказывать  

В результате работы должен получиться:  
- Jupyter-ноутбук с моделью  
- Признаки, влияющие больше всего на уровень продаж  
- Датасет, если после ваших манипуляций он отличается от исходного;  
- Документ с обоснованием решения и краткими результатами: какие техники и почему использовали, что получили, что можно улучшить (можно в рамках jupyter notebook’а)

Баллы	Что надо сделать  
10	Провести EDA  
10	Обработать категориальные признаки   
10	Устранить пропущенные значения  
10	Изучить корреляцию признаков с данными о продажах  
10	Выбрать и обосновать метрику, на основе которой будем измерять качество полученной модели  
20	Построить и подобрать оптимальные параметры для любой линейной модели  
20	Построить и подобрать оптимальные параметры для любой нелинейной модели  
20	Провести стекинг нескольких моделей  
10	Оценить качество модели на отложенной выборке  
10	Выбрать топ 3 признака больше всего влияющие на объемы продаж  

Все шаги решения должны сопровождаться подробным описанием полученных результатов и обоснованием выбора того или иного шага!  
За отсутствие описания так же будут снижаться баллы.  

Максимальное доступное количество баллов - 130  
Для получения зачета надо набрать минимум 80 баллов  
Для получения зачета с отличием надо набрать минимум 120 баллов  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

plt.rcParams["figure.figsize"] = [16, 9]

In [ ]:
data = pd.read_csv('data.csv')
data.head()

# Провести EDA

In [ ]:
# посмотрим глазами основные характеристики датасета

In [ ]:
data.info()

In [ ]:
data.describe().T

In [ ]:
columns_cat = data.select_dtypes(object).columns
columns_digit = data.select_dtypes(exclude=object).columns

print(f'Количество категориальных переменных: {len(columns_cat)}')
print(f'Количество количественных переменных: {len(columns_digit)}')

In [ ]:
# всего в датасете 8523 записей
# среднее по целевому признаку составляет 2 181,29; при этом есть очень большо разбор - от 33,29 до 13 086,96

# есть странный min=0 у признака Item_Visibility, а может и не странный
# есть нулевые значения в признаках Item_Weight, Outlet_Size
# много категорильных переменных, необходимо с ними поработать

# Устранить пропущенные значения

In [ ]:
# Item_Weight

In [ ]:
data[data.Item_Weight.isna() == 1].count()
data[data.Item_Weight.isna() == 1]

# 1463 нулевых значения; 6113 ненулевых значения; 
# гипотезы:
# - это продукты, вес которых реально очень низок. Можно уставить нулевыи или заполнить отрицательным значением
# - значение на самом деле есть, но по какой-то причине отсутствуют. Необходимо проставить или предсказать значения признака

In [ ]:
data[data.Item_Weight.isna() == 0]

In [ ]:
x = data[data.Item_Weight.isna() == 1].Item_Type.value_counts()

In [ ]:
y = data[data.Item_Weight.isna() == 0].Item_Type.value_counts()

In [ ]:
vc = pd.merge(pd.DataFrame(y), pd.DataFrame(x), how='outer', left_index=True, right_index=True)

In [ ]:
vc.plot.bar(rot=0)

In [ ]:
# кардинального различия по группам продуктов между продуктами со значениями и без нет. 
# скорее всего вес у этих позиций какой-то все же есть, либо он просто менее имеющегося "минимума" 4,555 по признаку, но 
# утверждать это нельзя. 

# Можно пробовать заполнить нули через предсказания отдельной модели

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [ ]:
# создадим копию датафрейма для работа с моделью

In [ ]:
tmp_df = data.copy()

In [ ]:
tmp_df_cat = tmp_df[columns_cat].astype(str)
tmp_df_digit = tmp_df[columns_digit]

In [ ]:
le = LabelEncoder()

In [ ]:
tmp_df_cat = tmp_df_cat.apply(le.fit_transform)

In [ ]:
tmp_df = tmp_df_digit.join(tmp_df_cat)

In [ ]:
# предварительно отделим категориальные переменные от количественных
# так как передать в модель текустовые признаки нельзя, то используем LableEncoder для перевода текста в цифру.

In [ ]:
# разобьем датасет на две части - по которой будем предсказывать вес и на которой будем обучаться. 
# Отдельно убираем признак Outlet_Size, так как в нем есть нули и мы хотим их отдельно предсказывать. 

iw_null = tmp_df[tmp_df.Item_Weight.isna() == 1].drop(['Item_Weight','Outlet_Size'], axis=1)

iw_data = tmp_df[tmp_df.Item_Weight.isna() == 0].drop(['Item_Weight','Outlet_Size'], axis=1)
iw_data_target = tmp_df[tmp_df.Item_Weight.isna() == 0].Item_Weight

In [ ]:
# отдельно разобьем "рабочие" данные на обучающую и тестовую выборки 
X_train, X_test, y_train, y_test  = train_test_split(iw_data, iw_data_target, test_size=0.25, random_state=42)

In [ ]:
# в качестве предсказательной модели возьмем RF из-за его лояльности к отсутствию OHE признаков.  
rf_iw = RandomForestRegressor(n_estimators=500,n_jobs=-1)

In [ ]:
rf_iw.fit(X_train, y_train)

In [ ]:
predictions_iw = rf_iw.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, predictions_iw))

# модель отработала; получили довольно низкий RMSE при средней по признаку 12,85. Прескажем реальные веса при помощи модели

In [ ]:
predictions_iw_real = rf_iw.predict(iw_null)

In [ ]:
tmp_df.Item_Weight.loc[iw_null.index] = predictions_iw_real
data.Item_Weight.loc[iw_null.index] = predictions_iw_real

In [ ]:
### построили модель RFRegressor`а для предсказания нулевых значений весов и заполнили их. 
# предположим, что это лучше, чем просто угадывать или заполнять нулями пустые значения. 

In [ ]:
# Outlet_Size

In [ ]:
data[data.Outlet_Size.isna() == 1].count()
data[data.Outlet_Size.isna() == 1]

# 2410 нулевых значения; 6 113 ненулевых значения; 
# гипотезы:
# - это островки, которые реально не имеют признака. Можно уставить нулевыи или заполнить отрицательным значением
# - значение на самом деле есть, но по какой-то причине отсутствуют. Необходимо предсказать значения признака

In [ ]:
# поопробуем посмотреть различия по Outlet_Location_Type и Outlet_Type

In [ ]:
x = data[data.Outlet_Size.isna() == 1].Outlet_Location_Type.value_counts()

In [ ]:
y = data[data.Outlet_Size.isna() == 0].Outlet_Location_Type.value_counts()

In [ ]:
pd.merge(pd.DataFrame(y), pd.DataFrame(x), how='outer', left_index=True, right_index=True).plot.bar(rot=0)

In [ ]:
x = data[data.Outlet_Size.isna() == 1].Outlet_Type.value_counts()

In [ ]:
y = data[data.Outlet_Size.isna() == 0].Outlet_Type.value_counts()

In [ ]:
pd.merge(pd.DataFrame(y), pd.DataFrame(x), how='outer', left_index=True, right_index=True).plot.bar(rot=0)

In [ ]:
### явно сказать, что отсутствующие признаки относятся к какому-то конкретному типу нельзя
# попробуем построить классификатор для определения этого признака
# тиакже воспользуемся моделью RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
os_null = tmp_df[tmp_df.Outlet_Size == 3].drop(['Item_Weight','Outlet_Size'], axis=1)

os_data = tmp_df[tmp_df.Outlet_Size != 3].drop(['Item_Weight','Outlet_Size'], axis=1)
os_data_target = tmp_df[tmp_df.Outlet_Size != 3].Outlet_Size

In [ ]:
from sklearn.preprocessing import label_binarize

In [ ]:
# для подсчета метрики качества используем label_binarize на целевую переменную

os_data_target = label_binarize(os_data_target, classes=[0, 1, 2])
n_classes = os_data_target.shape[1]

In [ ]:
X_train, X_test, y_train, y_test  = train_test_split(os_data, os_data_target, test_size=0.25, random_state=42)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
# для работы с несколькими классами применим OneVsRestClassifier, в качестве предсказывающей модели RF 

clf_os = OneVsRestClassifier(RandomForestClassifier(n_estimators=100,n_jobs=-1))
y_score = clf_os.fit(X_train, y_train).predict_proba(X_test)

In [ ]:
# нужно оценить получившиеся результаты, в качестве удобного и понятного средства оценки используем ROC AUC
# и ниже начались танцы с расчетом и визуализацией ROC_AUC для мультиклассовой классификации...

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve, auc
from scipy import interp
from itertools import cycle

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [ ]:
y_test == y_score

In [ ]:
### кривая явно либо ошибочная, либо говорит о переобучении, но не смог прежположить каой-то лучшй способ измерить. :(
### в итоге оставим эту модель для заполнения нулей

In [ ]:
predictions_os_real = np.argmax(clf_os.predict(os_null), axis=1)

In [ ]:
tmp_df.Outlet_Size.loc[os_null.index] = predictions_os_real
data.Outlet_Size.loc[os_null.index] = predictions_os_real

In [ ]:
# категорий мало, поэтому явно вернем их в изначальный текст

data.Outlet_Size = np.where(data['Outlet_Size'] == 0, 'High', 
                        np.where(data['Outlet_Size'] == 1, 'Medium', 
                        np.where(data['Outlet_Size'] == 2, 'Small', data['Outlet_Size'])))

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
# нулевые значения заполнены при помощи модели регрессирова и модели классификатора
# возвращен оригинальный датасет с категориальными признаками

# Обработать количественные признаки

In [ ]:
columns_digit

In [ ]:
df_cont = data[columns_digit]

In [ ]:
# большинство количественных признаков выглядит нормально, нулевых значений нет. Тем не менее, почти по каждому признаку 
# очень много уникальных значений. Это можно создать сложности для обучения модели. Попробуем округлить незначащие сотые 
# или тысячные и посмотреть на изменения пространства признаков 

In [ ]:
# Item_Weight
df_cont.Item_Weight.value_counts()

In [ ]:
sns.distplot(df_cont.Item_Weight.value_counts())

In [ ]:
df_cont[df_cont.Item_Weight < 10].Item_Weight.value_counts()

In [ ]:
# из-за точного веса в граммах получается очень большое количество уникальных признаков. 
# При этом при округлении их количество можно значитльно снизить. 

In [ ]:
df_cont.Item_Weight = df_cont.Item_Weight.round(0)

In [ ]:
# Item_Visibility
df_cont.Item_Visibility.value_counts()

In [ ]:
sns.distplot(df_cont.Item_Visibility.value_counts())

In [ ]:
df_cont.Item_Visibility.round(2).value_counts()

In [ ]:
df_cont.Item_Visibility = df_cont.Item_Visibility.round(2)
# при помощи округления удается сузить пространство признака до 34 разновидностей

In [ ]:
#Item_MRP
df_cont.Item_MRP.value_counts()

In [ ]:
sns.distplot(df_cont.Item_MRP.value_counts())

In [ ]:
df_cont.Item_MRP.round(0).value_counts()

In [ ]:
df_cont.Item_MRP = df_cont.Item_MRP.round(0)

In [ ]:
# Outlet_Establishment_Year
df_cont.Outlet_Establishment_Year.value_counts()

In [ ]:
sns.distplot(df_cont.Outlet_Establishment_Year.value_counts())

In [ ]:
# значений немного, логически они выглядят нормальными. Не будем трогать признак

In [ ]:
# Item_Outlet_Sales
df_cont.Item_Outlet_Sales.value_counts()

In [ ]:
sns.distplot(df_cont.Item_Outlet_Sales.value_counts())

In [ ]:
# мы не знаем в каких единицах дана информация, поэтому значения после запятой в продажах могут быть значительны. 
# так как это наша целевая переменная, то стараемся сохранить максимум информации и не производить трансформацию.

In [ ]:
df_cont

# Обработать категориальные признаки

In [ ]:
columns_cat

In [ ]:
data.info()

In [ ]:
data.Item_Identifier.value_counts()

In [ ]:
data.Item_Fat_Content.value_counts()

In [ ]:
data.Item_Type.value_counts()

In [ ]:
data.Outlet_Identifier.value_counts()

In [ ]:
data.Outlet_Size.value_counts()

In [ ]:
data.Outlet_Location_Type.value_counts()

In [ ]:
data.Outlet_Type.value_counts()

In [ ]:
# с виду с признаками все нормально. 
# Для деревьев можно сделать только OneHotEncoding, но для оценки также другими алгоритмами при помощи get dummies 
# преобразуем категории

In [ ]:
df_cat = pd.get_dummies(data[columns_cat])

# Изучить корреляцию признаков с данными о продажах

In [ ]:
# для оценки корреляции между признаками используем визуализированную матрицу корреляций
# в качестве данных возьмем датасет с предыдущего шага до любого рода get_dummies или ohe признаков. 

sns.heatmap(tmp_df.corr(), annot=True, cmap='RdYlGn', linewidths=0.1)
fig=plt.gcf()
fig.set_size_inches(35,35)
plt.show()

In [ ]:
# Коррекляции в данных есть. Между целевой переменной и item mrp, между другими пермененными. Тем не менее, 
# некоторые признаки действительно логически пересекаются, но не взаимоисключаемы. Нельзя сказатЬ, что какой-то 
# признак явно вычисляется через другие. Оставим их. 

In [ ]:
df = df_cont.join(df_cat)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# получили итоговый df для работы с предсказаниями

### Выбрать и обосновать метрику, на основе которой будем измерять качество полученной модели

In [ ]:
# Решаем задачу регрессии, поэтому можно для оценки выбирать один из показателей отклонения остатков - MAE, MSE, RMSE. 
# Отдельно мне хотелось бы избежать крупных ошибок модели, поэтому в качестве основного используем показатель RMSE.
# тем не менее, отдельно посчитаем MAE

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
# выбрали выборку для обучения, выбрали целевую переменную

X = df.drop('Item_Outlet_Sales', axis=1)
y = df['Item_Outlet_Sales']

In [ ]:
# разобьем данные на отложенную выборку и обучающую
X_train, X_val, y_train, y_val  = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# отдельно обучающую выборку разобьем на трейн и тест
X_train, X_test, y_train, y_test  = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Построить и подобрать оптимальные параметры для любой линейной модели

In [ ]:
# для подбора оптимальных параметров (лучшей комбинации) используем GridSearchCV
# итоговое поле признаков достаточно велико. Какие-то возможно не являются значимыми, поэтому в качестве линейной модели 
# используем Lasso регрессию с регуляризацией, которая зануляет коэффициенты перед наименее важными переменными 

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
params_Lasso = {
    'fit_intercept' : [False, True],
    'normalize' : [False, True]
    }

In [ ]:
grid_Lasso = GridSearchCV(Lasso(), param_grid=params_Lasso, cv=5, scoring='neg_mean_squared_error')
grid_Lasso.fit(X_train, y_train)

In [ ]:
print(grid_Lasso.best_params_)
print('RMSE: {}'.format(np.sqrt(np.abs(grid_Lasso.best_score_))))

In [ ]:
# что-то получилось. Простое среднее по признаку 2181.28, наша ошибка как правило меньше. Посмотрим другие модели. 

# Построить и подобрать оптимальные параметры для любой нелинейной модели

In [ ]:
# в качестве нелинейной модели я выбрал SVR из-за возможности попробовать разные ядра и попытаться найти наилучшее

In [ ]:
from sklearn.svm import SVR

In [ ]:
params_svr = {
    'kernel' : ['linear', 'poly', 'rbf', 'sigmoid']
}

In [ ]:
grid_svr = GridSearchCV(SVR(), param_grid=params_svr, cv=5, scoring='neg_mean_squared_error')
grid_svr.fit(X_train, y_train)

In [ ]:
print(grid_svr.best_params_)
print('RMSE: {}'.format(np.sqrt(np.abs(grid_svr.best_score_))))

In [ ]:
# результат подбора SVR параметров - линейное ядро, отсюда похожая оценки работа модели. 

# Провести стекинг нескольких моделей

In [ ]:
# для реализации стекинга используем штатный алгоритм StackingRegressor и наполним его LR, SVR, KNN моделями, в качестве 
# решающего применим RandomForestRegressor

In [ ]:
from sklearn.ensemble import StackingRegressor

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
classifier = StackingRegressor(
    [
        ('lr', LinearRegression()),
        ('svm', SVR()),
        ('knn',KNeighborsRegressor())
    ],
RandomForestRegressor())

In [ ]:
classifier.fit(X_train, y_train)

In [ ]:
classifier.named_estimators_['knn']

In [ ]:
y_pred_lr = classifier.named_estimators_['lr'].predict(X_test)
y_pred_svm = classifier.named_estimators_['svm'].predict(X_test)
y_pred_knn = classifier.named_estimators_['knn'].predict(X_test)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
print("RMSE lr: \t", round(np.sqrt(mean_squared_error(y_test, y_pred_lr)),2))
print("RMSE svm: \t", round(np.sqrt(mean_squared_error(y_test, y_pred_svm)),2))
print("RMSE classifier: \t", round(np.sqrt(mean_squared_error(y_test, y_pred)),2))

# Оценить качество модели на отложенной выборке

In [ ]:
# предскажем значения на отложенной выборке и оценим метрику качестве
y_pred_val_lr = classifier.named_estimators_['lr'].predict(X_val)
y_pred_val_svm = classifier.named_estimators_['svm'].predict(X_val)
y_pred_val_knn = classifier.named_estimators_['knn'].predict(X_val)

In [ ]:
y_pred_val = classifier.predict(X_val)

In [ ]:
print("RMSE lr: \t", round(np.sqrt(mean_squared_error(y_val, y_pred_val_lr)),2))
print("RMSE svm: \t", round(np.sqrt(mean_squared_error(y_val, y_pred_val_svm)),2))
print("RMSE KNN classifier: \t", round(np.sqrt(mean_squared_error(y_val, y_pred_val_knn)),2))

In [ ]:
plt.plot(y_val.index, y_val, 'o', markersize = 5)
plt.plot(y_val.index, y_pred_val, 'y^', markersize = 3)
plt.title('Правильные и предсказанные значения продаж')
plt.show()

In [ ]:
classifier.final_estimator_

In [ ]:
plt.barh(np.arange(len(classifier.final_estimator_.feature_importances_)),
                 classifier.final_estimator_.feature_importances_)
plt.yticks(np.arange(len(classifier.final_estimator_.feature_importances_)),classifier.estimators_)
''

In [ ]:
# наибольшее влияние при выьоре итогового предсказания оказывает KNN, при этом по показателю ошибки LinearRegression
# показывает просто катастрофу. Изменение/донастройка/замена этой модели из стекинга может улучшить или значительно изменить
# качество предсказания

# Выбрать топ 3 признака больше всего влияющие на объемы продаж